# Fundamental Trend Analysis
This notebook contains code for extracting fundamental data from an API and decomposing the series using the statsmodels library.

In [1]:
import pandas as pd
from datetime import date
from datetime import timedelta
import plotly.graph_objects as go

import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import numpy as np
from IPython.display import clear_output
import time
# Set plot space as inline for inline plots and qt for external plots
%matplotlib inline

In [2]:
# Obtain the API credentials from an external hidden file

from fmppassword import api_key

In [3]:
# Obtain today's date
today = date.today()

## 1. Obtain the Raw Data From the API

In [4]:
# A function to obtain fundamental data over time

#Utilizing "UniBit API" by Vincent Jiang
#Site: https://unibit.ai/product

#Errors may occur if credits run out, add an error code check to this function

def retrieve_ticker_fundamental_info(statement_type, ticker, interval):
    
    # A condition to set the interval
    if interval == "Q":
        interval = "?period=quarter"
    else:
        interval = None
    
    url = 'https://financialmodelingprep.com/api/v3/{}/{}{}&apikey={}'.format(statement_type, ticker, interval, api_key)
    response = requests.get(url)
    
    if response.status_code != 200:
        print("The response status code is: " + str(response.status_code))
        return None
    
    return response.json()

In [5]:
ticker = 'LOW'

# Interval can be Annual (A), Quarterly (Q), or Trailing 12 Month (TTM)
interval = 'Q'

# Statement type can be: "income-statement-as-reported" (Income Statement),
#                        "balance-sheet-statement-as-reported" (Balance Sheet),
#                        "cash-flow-statement-as-reported" (Cash Flow),
#                        or "financial-statement-as-reported" (All Statements).
statement_type = 'income-statement-as-reported'

fundamental_data = retrieve_ticker_fundamental_info(statement_type, ticker, interval)

In [7]:
def retrieve_fundamental_data_dataframe(ticker, fundamental_data):
    
    data_frame = pd.DataFrame(fundamental_data)
    
    # Set the index to the 'date' column
    data_frame.set_index('date', inplace = True)
    
    data_frame.sort_index(inplace=True)
    
    return data_frame

In [22]:
low_df = retrieve_fundamental_data_dataframe(ticker, fundamental_data)
#print(low_df.columns)
low_df[['symbol', 'period','interestrevenueexpensenet', 'revenuefromcontractwithcustomerexcludingassessedtaxpercentage', 'revenuefromcontractwithcustomerexcludingassessedtax', 'operatingexpenses', 'salesrevenuenet', 'salesrevenuenetpercent']]

,symbol,period,interestrevenueexpensenet,revenuefromcontractwithcustomerexcludingassessedtaxpercentage,revenuefromcontractwithcustomerexcludingassessedtax,operatingexpenses,salesrevenuenet,salesrevenuenetpercent
date,,,,,,,,
2010-01-29,LOW,FY,NaN,NaN,NaN,1.363800e+10,4.722000e+10,1.0
2011-10-28,LOW,Q3,-91000000.0,NaN,NaN,3.684000e+09,1.185100e+10,0.0
2012-02-03,LOW,FY,-102000000.0,NaN,NaN,3.494000e+09,1.162900e+10,0.0
2012-05-04,LOW,Q1,-103000000.0,NaN,NaN,3.714000e+09,1.315300e+10,1.0
2012-08-03,LOW,Q2,-96000000.0,NaN,NaN,3.638000e+09,1.424900e+10,0.0
2012-11-02,LOW,Q3,-114000000.0,NaN,NaN,3.508000e+09,1.207300e+10,0.0
2013-02-01,LOW,FY,-110000000.0,NaN,NaN,3.330000e+09,1.104600e+10,0.0
2013-05-03,LOW,Q1,-113000000.0,NaN,NaN,3.687000e+09,1.308800e+10,1.0
2013-08-02,LOW,Q2,-110000000.0,NaN,NaN,3.890000e+09,1.571200e+10,0.0
